In [ ]:
from sklearn.metrics.pairwise import rbf_kernel, sigmoid_kernel, polynomial_kernel
from tempfile import TemporaryFile
import pandas as pd
from itertools import product
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from sklearn.preprocessing import KernelCenterer
from typing import Any, Dict, Union, Tuple, List
from pprint import pprint
import warnings
warnings.filterwarnings('always')
import sys
maindir = '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(maindir)
from source.utils import make_symmetric_matrix_psd, normalize, make_kernel_combinations
from source.utils import multitask, make_kernel_matrix

## Calculate kernel matrices per parameter and kernel combination  

In [ ]:
kerneldir = os.path.join(maindir, 'data/precomputed_multitask_kernels')
input_file = os.path.join(maindir, "data/kernel_parameter.csv")
kernel_param = np.array(pd.read_csv(input_file))
kernel_param

## Make kernel parameter combinations 

In [ ]:
kernel_comb_sig_rbf_rbf = make_kernel_combinations(kernel_param, "sigmoid_kernel", "rbf_kernel", "rbf_kernel")
kernel_comb_sig_rbf_rbf

In [ ]:
kernel_comb_rbf_rbf_rbf = make_kernel_combinations(kernel_param, "rbf_kernel", "rbf_kernel", "rbf_kernel")
kernel_comb_rbf_rbf_rbf

In [ ]:
kernel_comb_sig_poly_poly = make_kernel_combinations(kernel_param, "sigmoid_kernel", "poly_kernel", "poly_kernel")
kernel_comb_sig_poly_poly

In [ ]:
kernel_comb_rbf_poly_poly = make_kernel_combinations(kernel_param, "rbf_kernel", "poly_kernel", "poly_kernel")
kernel_comb_rbf_poly_poly

In [ ]:
kernel_comb_sig_rbf_poly = make_kernel_combinations(kernel_param, "sigmoid_kernel", "rbf_kernel", "poly_kernel")
kernel_comb_sig_rbf_poly

In [ ]:
kernel_comb_rbf_rbf_poly = make_kernel_combinations(kernel_param, "rbf_kernel", "rbf_kernel", "poly_kernel")
kernel_comb_rbf_rbf_poly

In [ ]:
kernel_comb_sig_poly_rbf = make_kernel_combinations(kernel_param, "sigmoid_kernel", "poly_kernel", "rbf_kernel")
kernel_comb_sig_poly_rbf

In [ ]:
kernel_comb_rbf_poly_rbf = make_kernel_combinations(kernel_param, "rbf_kernel", "poly_kernel", "rbf_kernel")
kernel_comb_rbf_poly_rbf

### Save kernel parameter combinations as grid

In [ ]:
fn = os.path.join(maindir, 'data/precomputed_multitask_kernels/kernel_comb_SRR.json')
with open(fn, 'w') as fp:
    json.dump({k: v.tolist() for k, v in kernel_comb_sig_rbf_rbf.items()}, fp)

In [ ]:
fn = os.path.join(maindir, 'data/precomputed_multitask_kernels/kernel_comb_RRR.json')
with open(fn, 'w') as fp:
    json.dump({k: v.tolist() for k, v in kernel_comb_rbf_rbf_rbf.items()}, fp)

In [ ]:
fn = os.path.join(maindir, 'data/precomputed_multitask_kernels/kernel_comb_SPP.json')
with open(fn, 'w') as fp:
    json.dump({k: v.tolist() for k, v in kernel_comb_sig_poly_poly.items()}, fp)

In [ ]:
fn = os.path.join(maindir, 'data/precomputed_multitask_kernels/kernel_comb_RPP.json')
with open(fn, 'w') as fp:
    json.dump({k: v.tolist() for k, v in kernel_comb_rbf_poly_poly.items()}, fp)

In [ ]:
fn = os.path.join(maindir, 'data/precomputed_multitask_kernels/kernel_comb_SRP.json')
with open(fn, 'w') as fp:
    json.dump({k: v.tolist() for k, v in kernel_comb_sig_rbf_poly.items()}, fp)

In [ ]:
fn = os.path.join(maindir, 'data/precomputed_multitask_kernels/kernel_comb_RRP.json')
with open(fn, 'w') as fp:
    json.dump({k: v.tolist() for k, v in kernel_comb_rbf_rbf_poly.items()}, fp)

In [ ]:
fn = os.path.join(maindir, 'data/precomputed_multitask_kernels/kernel_comb_SPR.json')
with open(fn, 'w') as fp:
    json.dump({k: v.tolist() for k, v in kernel_comb_sig_poly_rbf.items()}, fp)

In [ ]:
fn = os.path.join(maindir, 'data/precomputed_multitask_kernels/kernel_comb_RPR.json')
with open(fn, 'w') as fp:
    json.dump({k: v.tolist() for k, v in kernel_comb_rbf_poly_rbf.items()}, fp)

## Make kernel matrices with different kernel combinations

## Whole proteome data

In [ ]:
input_data = os.path.join(maindir, "data/proteome_data/preprocessed_whole_data.csv")
data_malaria = pd.read_csv(input_data)
data_malaria.head()

## Selective proteome data set

In [ ]:
#load file via pandas as csv file 
input_file_s = os.path.join(maindir, "data/proteome_data/preprocessed_selective_data.csv")
selected_data_malaria = pd.read_csv(input_file_s)
selected_data_malaria.head()

In [ ]:
def main(
    identifier: str,
    data: pd.DataFrame,
    kernel_param: dict,
    kernel_for_time_series: str,
    kernel_for_dosage: str,
    kernel_for_abSignal: str,
    save_to_dir: str,
    scale: bool = False,
) -> Dict[str, List[Any]]:

    collection = dict()
    collection['models'] = []
    collection['kernel_matrices'] = []
    collection['information'] = []
    collection['damping_values_list'] = []
    if kernel_for_time_series == "sigmoid_kernel" and kernel_for_dosage == "rbf_kernel" and kernel_for_abSignal == "rbf_kernel":
        for m in product(kernel_param["SA"], kernel_param["SO"], kernel_param["R0"], kernel_param["R1"], kernel_param["R2"], kernel_param["P1"], kernel_param["P2"]):
            kernel_matrix, damping_values, info = make_kernel_matrix(
                AB_signals=data.drop(columns=['Patient', 'group', 'Protection', 'TimePointOrder', 'Dose']),
                time_series=data['TimePointOrder'],
                dose=data['Dose'],
                model=m,
                kernel_time_series=kernel_for_time_series,
                kernel_dosage=kernel_for_dosage,
                kernel_abSignals=kernel_for_abSignal,
                scale=scale,
            )
            collection['models'].append(m)
            collection['kernel_matrices'].append(kernel_matrix)
            collection['information'].append(info)
            collection['damping_values_list'].append(damping_values)
            output_file_name = f"{identifier}_SRR_SA_{m[0]}_SO_{m[1]}_R0_{m[2]}_R1_{m[3]:.1E}_R2_{m[4]:.1E}_P1_{m[5]}_P2_{m[6]}.npy"
            np.save(os.path.join(save_to_dir, output_file_name), kernel_matrix)

    elif kernel_for_time_series == "sigmoid_kernel" and kernel_for_dosage == "poly_kernel" and kernel_for_abSignal == "rbf_kernel":
        for m in product(kernel_param["SA"], kernel_param["SO"], kernel_param["R0"], kernel_param["R1"], kernel_param["R2"], kernel_param["P1"], kernel_param["P2"]):
            kernel_matrix, damping_values, info = make_kernel_matrix(
                AB_signals=data.drop(columns=['Patient', 'group', 'Protection', 'TimePointOrder', 'Dose']),
                time_series=data['TimePointOrder'],
                dose=data['Dose'],
                model=m,
                kernel_time_series=kernel_for_time_series,
                kernel_dosage=kernel_for_dosage,
                kernel_abSignals=kernel_for_abSignal,
                scale=scale,
            )
            collection['models'].append(m)
            collection['kernel_matrices'].append(kernel_matrix)
            collection['information'].append(info)
            collection['damping_values_list'].append(damping_values)
            output_file_name = f"{identifier}_SPR_SA_{m[0]}_SO_{m[1]}_R0_{m[2]}_R1_{m[3]}_R2_{m[4]:.1E}_P1_{m[5]}_P2_{m[6]}.npy"
            np.save(os.path.join(save_to_dir, output_file_name), kernel_matrix)
            
    elif kernel_for_time_series == "sigmoid_kernel" and kernel_for_dosage == "rbf_kernel" and kernel_for_abSignal == "poly_kernel":
        for m in product(kernel_param["SA"], kernel_param["SO"], kernel_param["R0"], kernel_param["R1"], kernel_param["R2"], kernel_param["P1"], kernel_param["P2"]):
            kernel_matrix, damping_values, info = make_kernel_matrix(
                AB_signals=data.drop(columns=['Patient', 'group', 'Protection', 'TimePointOrder', 'Dose']),
                time_series=data['TimePointOrder'],
                dose=data['Dose'],
                model=m,
                kernel_time_series=kernel_for_time_series,
                kernel_dosage=kernel_for_dosage,
                kernel_abSignals=kernel_for_abSignal,
                scale=scale,
            )
            collection['models'].append(m)
            collection['kernel_matrices'].append(kernel_matrix)
            collection['information'].append(info)
            collection['damping_values_list'].append(damping_values)
            output_file_name = f"{identifier}_SRP_SA_{m[0]}_SO_{m[1]}_R0_{m[2]}_R1_{m[3]:.1E}_R2_{m[4]}_P1_{m[5]}_P2_{m[6]}.npy"
            np.save(os.path.join(save_to_dir, output_file_name), kernel_matrix)
            
    elif kernel_for_time_series == "sigmoid_kernel" and kernel_for_dosage == "poly_kernel" and kernel_for_abSignal == "poly_kernel":
        for m in product(kernel_param["SA"], kernel_param["SO"], kernel_param["R0"], kernel_param["R1"], kernel_param["R2"], kernel_param["P1"], kernel_param["P2"]):
            kernel_matrix, damping_values, info = make_kernel_matrix(
                AB_signals=data.drop(columns=['Patient', 'group', 'Protection', 'TimePointOrder', 'Dose']),
                time_series=data['TimePointOrder'],
                dose=data['Dose'],
                model=m,
                kernel_time_series=kernel_for_time_series,
                kernel_dosage=kernel_for_dosage,
                kernel_abSignals=kernel_for_abSignal,
                scale=scale,
            )
            collection['models'].append(m)
            collection['kernel_matrices'].append(kernel_matrix)
            collection['information'].append(info)
            collection['damping_values_list'].append(damping_values)
            output_file_name = f"{identifier}_SPP_SA_{m[0]}_SO_{m[1]}_R0_{m[2]}_R1_{m[3]}_R2_{m[4]}_P1_{m[5]}_P2_{m[6]}.npy"
            np.save(os.path.join(save_to_dir, output_file_name), kernel_matrix)
    elif kernel_for_time_series == "rbf_kernel" and kernel_for_dosage == "rbf_kernel" and kernel_for_abSignal == "rbf_kernel":
        for m in product(kernel_param["SA"], kernel_param["SO"], kernel_param["R0"], kernel_param["R1"], kernel_param["R2"], kernel_param["P1"], kernel_param["P2"]):
            kernel_matrix, damping_values, info = make_kernel_matrix(
                AB_signals=data.drop(columns=['Patient', 'group', 'Protection', 'TimePointOrder', 'Dose']),
                time_series=data['TimePointOrder'],
                dose=data['Dose'],
                model=m,
                kernel_time_series=kernel_for_time_series,
                kernel_dosage=kernel_for_dosage,
                kernel_abSignals=kernel_for_abSignal,
                scale=scale,
            )
            collection['models'].append(m)
            collection['kernel_matrices'].append(kernel_matrix)
            collection['information'].append(info)
            collection['damping_values_list'].append(damping_values)
            output_file_name = f"{identifier}_RRR_SA_{m[0]}_SO_{m[1]}_R0_{m[2]:.1E}_R1_{m[3]:.1E}_R2_{m[4]:.1E}_P1_{m[5]}_P2_{m[6]}.npy"
            np.save(os.path.join(save_to_dir, output_file_name), kernel_matrix)

    elif kernel_for_time_series == "rbf_kernel" and kernel_for_dosage == "poly_kernel" and kernel_for_abSignal == "rbf_kernel":
        for m in product(kernel_param["SA"], kernel_param["SO"], kernel_param["R0"], kernel_param["R1"], kernel_param["R2"], kernel_param["P1"], kernel_param["P2"]):
            kernel_matrix, damping_values, info = make_kernel_matrix(
                AB_signals=data.drop(columns=['Patient', 'group', 'Protection', 'TimePointOrder', 'Dose']),
                time_series=data['TimePointOrder'],
                dose=data['Dose'],
                model=m,
                kernel_time_series=kernel_for_time_series,
                kernel_dosage=kernel_for_dosage,
                kernel_abSignals=kernel_for_abSignal,
                scale=scale,
            )
            collection['models'].append(m)
            collection['kernel_matrices'].append(kernel_matrix)
            collection['information'].append(info)
            collection['damping_values_list'].append(damping_values)
            output_file_name = f"{identifier}_RPR_SA_{m[0]}_SO_{m[1]}_R0_{m[2]:.1E}_R1_{m[3]}_R2_{m[4]:.1E}_P1_{m[5]}_P2_{m[6]}.npy"
            np.save(os.path.join(save_to_dir, output_file_name), kernel_matrix)
            
    elif kernel_for_time_series == "rbf_kernel" and kernel_for_dosage == "rbf_kernel" and kernel_for_abSignal == "poly_kernel":
        for m in product(kernel_param["SA"], kernel_param["SO"], kernel_param["R0"], kernel_param["R1"], kernel_param["R2"], kernel_param["P1"], kernel_param["P2"]):
            kernel_matrix, damping_values, info = make_kernel_matrix(
                AB_signals=data.drop(columns=['Patient', 'group', 'Protection', 'TimePointOrder', 'Dose']),
                time_series=data['TimePointOrder'],
                dose=data['Dose'],
                model=m,
                kernel_time_series=kernel_for_time_series,
                kernel_dosage=kernel_for_dosage,
                kernel_abSignals=kernel_for_abSignal,
                scale=scale,
            )
            collection['models'].append(m)
            collection['kernel_matrices'].append(kernel_matrix)
            collection['information'].append(info)
            collection['damping_values_list'].append(damping_values)
            output_file_name = f"{identifier}_RRP_SA_{m[0]}_SO_{m[1]}_R0_{m[2]:.1E}_R1_{m[3]:.1E}_R2_{m[4]}_P1_{m[5]}_P2_{m[6]}.npy"
            np.save(os.path.join(save_to_dir, output_file_name), kernel_matrix)
            
    elif kernel_for_time_series == "rbf_kernel" and kernel_for_dosage == "poly_kernel" and kernel_for_abSignal == "poly_kernel":
        for m in product(kernel_param["SA"], kernel_param["SO"], kernel_param["R0"], kernel_param["R1"], kernel_param["R2"], kernel_param["P1"], kernel_param["P2"]):
            kernel_matrix, damping_values, info = make_kernel_matrix(
                AB_signals=data.drop(columns=['Patient', 'group', 'Protection', 'TimePointOrder', 'Dose']),
                time_series=data['TimePointOrder'],
                dose=data['Dose'],
                model=m,
                kernel_time_series=kernel_for_time_series,
                kernel_dosage=kernel_for_dosage,
                kernel_abSignals=kernel_for_abSignal,
                scale=scale,
            )
            collection['models'].append(m)
            collection['kernel_matrices'].append(kernel_matrix)
            collection['information'].append(info)
            collection['damping_values_list'].append(damping_values)
            output_file_name = f"{identifier}_RPP_SA_{m[0]}_SO_{m[1]}_R0_{m[2]:.1E}_R1_{m[3]}_R2_{m[4]}_P1_{m[5]}_P2_{m[6]}.npy"
            np.save(os.path.join(save_to_dir, output_file_name), kernel_matrix)

    return collection

In [ ]:
def heatmap2d(arr: np.ndarray):
    plt.figure(figsize=(9.5, 8))
    plt.imshow(arr, cmap='hot')
    plt.colorbar()
    plt.show()

In [ ]:
def overview(collection):
    damping_value_sums = []
    for i in collection['damping_values_list']:
        damping_value_sums.append(np.sum(i))
    print('maximum sum:', np.max(damping_value_sums))
    print('sorted sums:', np.sort(damping_value_sums)[::-1])
    print('')

    for i in range(10):
        print(collection['models'][i])
        print(np.sum(collection['damping_values_list'][i]))
        plt.plot(np.sort(np.linalg.eigvals(collection['kernel_matrices'][i]), kind='mergesort')[::-1], marker='.')
        plt.show()
        plt.close()
        heatmap2d(collection['kernel_matrices'][i])

In [ ]:
# SRR unscaled

collection_SRR = main(
    identifier='kernel_matrix',
    data=data_malaria,
    kernel_param=kernel_comb_sig_rbf_rbf,
    kernel_for_time_series="sigmoid_kernel",
    kernel_for_dosage="rbf_kernel",
    kernel_for_abSignal="rbf_kernel",
    save_to_dir=os.path.join(kerneldir, 'unscaled'),
)
fn = os.path.join(kerneldir, 'unfiltered/collected_damping_info_SRR.json')
with open(fn, 'w') as fp:
    json.dump({x: collection_SRR[x] for x in collection_SRR if x not in 'kernel_matrices'}, fp)

In [ ]:
overview(collection_SRR)

In [ ]:
# RRR unscaled

collection_RRR = main(
    identifier='kernel_matrix',
    data=data_malaria,
    kernel_param=kernel_comb_rbf_rbf_rbf,
    kernel_for_time_series="rbf_kernel",
    kernel_for_dosage="rbf_kernel",
    kernel_for_abSignal="rbf_kernel",
    save_to_dir=os.path.join(kerneldir, 'unscaled'),
)
fn = os.path.join(kerneldir, 'unfiltered/collected_damping_info_RRR.json')
with open(fn, 'w') as fp:
    json.dump({x: collection_RRR[x] for x in collection_RRR if x not in 'kernel_matrices'}, fp)

In [ ]:
overview(collection_RRR)

In [ ]:
# SRP unscaled

collection_SRP = main(
    identifier='kernel_matrix',
    data=data_malaria,
    kernel_param=kernel_comb_sig_rbf_poly,
    kernel_for_time_series="sigmoid_kernel",
    kernel_for_dosage="rbf_kernel",
    kernel_for_abSignal="poly_kernel",
    save_to_dir=os.path.join(kerneldir, 'unscaled'),
)
fn = os.path.join(kerneldir, 'unfiltered/collected_damping_info_SRP.json')
with open(fn, 'w') as fp:
    json.dump({x: collection_SRP[x] for x in collection_SRP if x not in 'kernel_matrices'}, fp)

In [ ]:
overview(collection_SRP)

In [ ]:
# RRP unscaled

collection_RRP = main(
    identifier='kernel_matrix',
    data=data_malaria,
    kernel_param=kernel_comb_rbf_rbf_poly,
    kernel_for_time_series="rbf_kernel",
    kernel_for_dosage="rbf_kernel",
    kernel_for_abSignal="poly_kernel",
    save_to_dir=os.path.join(kerneldir, 'unscaled'),
)
fn = os.path.join(kerneldir, 'unfiltered/collected_damping_info_RRP.json')
with open(fn, 'w') as fp:
    json.dump({x: collection_RRP[x] for x in collection_RRP if x not in 'kernel_matrices'}, fp)

In [ ]:
overview(collection_RRP)

In [ ]:
# SPR unscaled

collection_SPR = main(
    identifier='kernel_matrix',
    data=data_malaria,
    kernel_param=kernel_comb_sig_poly_rbf,
    kernel_for_time_series="sigmoid_kernel",
    kernel_for_dosage="poly_kernel",
    kernel_for_abSignal="rbf_kernel",
    save_to_dir=os.path.join(kerneldir, 'unscaled'),
)
fn = os.path.join(kerneldir, 'unfiltered/collected_damping_info_SPR.json')
with open(fn, 'w') as fp:
    json.dump({x: collection_SPR[x] for x in collection_SPR if x not in 'kernel_matrices'}, fp)

In [ ]:
overview(collection_SPR)

In [ ]:
# RPR unscaled

collection_RPR = main(
    identifier='kernel_matrix',
    data=data_malaria,
    kernel_param=kernel_comb_rbf_poly_rbf,
    kernel_for_time_series="rbf_kernel",
    kernel_for_dosage="poly_kernel",
    kernel_for_abSignal="rbf_kernel",
    save_to_dir=os.path.join(kerneldir, 'unscaled'),
)
fn = os.path.join(kerneldir, 'unfiltered/collected_damping_info_RPR.json')
with open(fn, 'w') as fp:
    json.dump({x: collection_RPR[x] for x in collection_RPR if x not in 'kernel_matrices'}, fp)

In [ ]:
overview(collection_RPR)

In [ ]:
# SPP unscaled

collection_SPP = main(
    identifier='kernel_matrix',
    data=data_malaria,
    kernel_param=kernel_comb_sig_poly_poly,
    kernel_for_time_series="sigmoid_kernel",
    kernel_for_dosage="poly_kernel",
    kernel_for_abSignal="poly_kernel",
    save_to_dir=os.path.join(kerneldir, 'unscaled'),
)
fn = os.path.join(kerneldir, 'unfiltered/collected_damping_info_SPP.json')
with open(fn, 'w') as fp:
    json.dump({x: collection_SPP[x] for x in collection_SPP if x not in 'kernel_matrices'}, fp)

In [ ]:
overview(collection_SPP)

In [ ]:
# RPP unscaled

collection_RPP = main(
    identifier='kernel_matrix',
    data=data_malaria,
    kernel_param=kernel_comb_rbf_poly_poly,
    kernel_for_time_series="rbf_kernel",
    kernel_for_dosage="poly_kernel",
    kernel_for_abSignal="poly_kernel",
    save_to_dir=os.path.join(kerneldir, 'unscaled'),
)
fn = os.path.join(kerneldir, 'unfiltered/collected_damping_info_RPP.json')
with open(fn, 'w') as fp:
    json.dump({x: collection_RPP[x] for x in collection_RPP if x not in 'kernel_matrices'}, fp)

In [ ]:
overview(collection_RPP)

In [ ]:
target_label_vec = data_malaria_sorted.loc[:, "Patient":"TimePointOrder"]

In [ ]:
target_label_vec.to_csv(os.path.join(kerneldir, "unfiltered/target_label_vec.csv"))

In [ ]:
# SelectiveSet SRR unscaled

collection_SRR = main(
    identifier='kernel_matrix_SelectiveSet',
    data=selected_data_malaria,
    kernel_param=kernel_comb_sig_rbf_rbf,
    kernel_for_time_series="sigmoid_kernel",
    kernel_for_dosage="rbf_kernel",
    kernel_for_abSignal="rbf_kernel",
    save_to_dir=os.path.join(kerneldir, 'unscaled'),
)
fn = os.path.join(kerneldir, 'unfiltered/collected_damping_info_SelectiveSet_SRR.json')
with open(fn, 'w') as fp:
    json.dump({x: collection_SRR[x] for x in collection_SRR if x not in 'kernel_matrices'}, fp)

In [ ]:
overview(collection_SRR)

In [ ]:
# SelectiveSet RRR unscaled

collection_RRR = main(
    identifier='kernel_matrix_SelectiveSet',
    data=selected_data_malaria,
    kernel_param=kernel_comb_rbf_rbf_rbf,
    kernel_for_time_series="rbf_kernel",
    kernel_for_dosage="rbf_kernel",
    kernel_for_abSignal="rbf_kernel",
    save_to_dir=os.path.join(kerneldir, 'unscaled'),
)
fn = os.path.join(kerneldir, 'unfiltered/collected_damping_info_SelectiveSet_RRR.json')
with open(fn, 'w') as fp:
    json.dump({x: collection_RRR[x] for x in collection_RRR if x not in 'kernel_matrices'}, fp)

In [ ]:
overview(collection_RRR)

In [ ]:
# SelectiveSet SRP unscaled

collection_SRP = main(
    identifier='kernel_matrix_SelectiveSet',
    data=selected_data_malaria,
    kernel_param=kernel_comb_sig_rbf_poly,
    kernel_for_time_series="sigmoid_kernel",
    kernel_for_dosage="rbf_kernel",
    kernel_for_abSignal="poly_kernel",
    save_to_dir=os.path.join(kerneldir, 'unscaled'),
)
fn = os.path.join(kerneldir, 'unfiltered/collected_damping_info_SelectiveSet_SRP.json')
with open(fn, 'w') as fp:
    json.dump({x: collection_SRP[x] for x in collection_SRP if x not in 'kernel_matrices'}, fp)

In [ ]:
overview(collection_SRP)

In [ ]:
# SelectiveSet RRP unscaled

collection_RRP = main(
    identifier='kernel_matrix_SelectiveSet',
    data=selected_data_malaria,
    kernel_param=kernel_comb_rbf_rbf_poly,
    kernel_for_time_series="rbf_kernel",
    kernel_for_dosage="rbf_kernel",
    kernel_for_abSignal="poly_kernel",
    save_to_dir=os.path.join(kerneldir, 'unscaled'),
)
fn = os.path.join(kerneldir, 'unfiltered/collected_damping_info_SelectiveSet_RRP.json')
with open(fn, 'w') as fp:
    json.dump({x: collection_RRP[x] for x in collection_RRP if x not in 'kernel_matrices'}, fp)

In [ ]:
overview(collection_RRP)

In [ ]:
# SelectiveSet SPR unscaled

collection_SPR = main(
    identifier='kernel_matrix_SelectiveSet',
    data=selected_data_malaria,
    kernel_param=kernel_comb_sig_poly_rbf,
    kernel_for_time_series="sigmoid_kernel",
    kernel_for_dosage="poly_kernel",
    kernel_for_abSignal="rbf_kernel",
    save_to_dir=os.path.join(kerneldir, 'unscaled'),
)
fn = os.path.join(kerneldir, 'unfiltered/collected_damping_info_SelectiveSet_SPR.json')
with open(fn, 'w') as fp:
    json.dump({x: collection_SPR[x] for x in collection_SPR if x not in 'kernel_matrices'}, fp)

In [ ]:
overview(collection_SPR)

In [ ]:
# SelectiveSet RPR unscaled

collection_RPR = main(
    identifier='kernel_matrix_SelectiveSet',
    data=selected_data_malaria,
    kernel_param=kernel_comb_rbf_poly_rbf,
    kernel_for_time_series="rbf_kernel",
    kernel_for_dosage="poly_kernel",
    kernel_for_abSignal="rbf_kernel",
    save_to_dir=os.path.join(kerneldir, 'unscaled'),
)
fn = os.path.join(kerneldir, 'unfiltered/collected_damping_info_SelectiveSet_RPR.json')
with open(fn, 'w') as fp:
    json.dump({x: collection_RPR[x] for x in collection_RPR if x not in 'kernel_matrices'}, fp)

In [ ]:
overview(collection_RPR)

In [ ]:
# SelectiveSet SPP unscaled

collection_SPP = main(
    identifier='kernel_matrix_SelectiveSet',
    data=selected_data_malaria,
    kernel_param=kernel_comb_sig_poly_poly,
    kernel_for_time_series="sigmoid_kernel",
    kernel_for_dosage="poly_kernel",
    kernel_for_abSignal="poly_kernel",
    save_to_dir=os.path.join(kerneldir, 'unscaled'),
)
fn = os.path.join(kerneldir, 'unfiltered/collected_damping_info_SelectiveSet_SPP.json')
with open(fn, 'w') as fp:
    json.dump({x: collection_SPP[x] for x in collection_SPP if x not in 'kernel_matrices'}, fp)

In [ ]:
overview(collection_SPP)

In [ ]:
# SelectiveSet RPP unscaled

collection_RPP = main(
    identifier='kernel_matrix_SelectiveSet',
    data=selected_data_malaria,
    kernel_param=kernel_comb_rbf_poly_poly,
    kernel_for_time_series="rbf_kernel",
    kernel_for_dosage="poly_kernel",
    kernel_for_abSignal="poly_kernel",
    save_to_dir=os.path.join(kerneldir, 'unscaled'),
)
fn = os.path.join(kerneldir, 'unfiltered/collected_damping_info_SelectiveSet_RPP.json')
with open(fn, 'w') as fp:
    json.dump({x: collection_RPP[x] for x in collection_RPP if x not in 'kernel_matrices'}, fp)

In [ ]:
overview(collection_RPP)